In [ ]:
import os
from collections import defaultdict
from typing import Any

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import colors as mcolors
from matplotlib import patches as mpatches

In [ ]:
if os.path.exists("../custom_style.mplstyle"):
    plt.style.use(["ggplot", "../custom_style.mplstyle"])

In [ ]:
figs_save_dir = os.path.join("../figures", "eval_metrics")
os.makedirs(figs_save_dir, exist_ok=True)

In [ ]:
WORK_DIR = os.getenv("WORK", "")
DATA_DIR = os.path.join(WORK_DIR, "data")

In [ ]:
data_split = "final_skew40/test_zeroshot"

run_names_chattn = {
    "chattn + mlm + embedding": "pft_stand_rff_only_pretrained",
    # "chattn + mlm + embedding": "pft_chattn_fullemb_pretrained",
    # "chattn + mlm + embedding": "pft_chattn_fullemb_quartic_enc",
    "chattn + mlm": "pft_chattn_noembed_pretrained_correct",
    "chattn + embedding": "pft_fullyfeat_from_scratch",
    # "chattn + embedding": "pft_chattn_emb_w_poly",
    # "chattn + embedding": "pft_fullyfeat_from_scratch_longer",
    "chattn": "pft_stand_chattn_noemb",
}

run_names_no_chattn = {
    # "univariate (wider) + embedding": "pft_emb_equal_param_univariate_from_scratch",
    "univariate (wider)": "pft_noemb_equal_param_univariate_from_scratch",
    "univariate (deeper)": "pft_equal_param_deeper_univariate_from_scratch_noemb",
    # "univariate + mlm + embedding": "pft_rff_univariate_pretrained",
    # "univariate + mlm": "pft_vanilla_pretrained_correct",
    # "tailmask": "pft_tailmasking_from_scratch",
}

run_names = {
    **run_names_chattn,
    **run_names_no_chattn,
}

run_metrics_dirs_all_groups = {
    "chattn": {
        run_abbrv: os.path.join(
            "../eval_results",
            "patchtst",
            f"{run_name}-0",
            data_split,
        )
        for run_abbrv, run_name in run_names_chattn.items()
    },
    "no_chattn": {
        run_abbrv: os.path.join(
            "../eval_results",
            "patchtst",
            f"{run_name}-0",
            data_split,
        )
        for run_abbrv, run_name in run_names_no_chattn.items()
    },
}

In [ ]:
run_names_to_use_sliding_context = {
    "chattn": run_names_chattn.keys(),
    "no_chattn": run_names_no_chattn.keys(),
}
for group, run_names in run_names_to_use_sliding_context.items():
    for run_name in run_names:
        curr_path = run_metrics_dirs_all_groups[group][run_name]
        new_path = curr_path.replace("patchtst", "patchtst_sliding")
        run_metrics_dirs_all_groups[group][run_name] = new_path

In [ ]:
run_metrics_dirs_all_groups

In [ ]:
run_metrics_dirs_all_groups["no_chattn"].keys()

In [ ]:
metrics_all = defaultdict(lambda: defaultdict(dict))
for run_group, run_metrics_dir_dict in run_metrics_dirs_all_groups.items():
    print(f"Run group: {run_group}")
    for run_abbrv, run_metrics_dir in run_metrics_dir_dict.items():
        run_abbrv = str(run_abbrv)
        print(f"{run_abbrv}: {run_metrics_dir}")
        for file in sorted(
            os.listdir(run_metrics_dir),
            key=lambda x: int(x.split("_pred")[1].split(".csv")[0]),
        ):
            if file.endswith(".csv"):
                prediction_length = int(file.split("_pred")[1].split(".csv")[0])
                # print(f"Prediction length: {prediction_length} for {run_abbrv}")
                with open(os.path.join(run_metrics_dir, file), "r") as f:
                    metrics = pd.read_csv(f).to_dict()
                    metrics_all[run_group][run_abbrv][prediction_length] = metrics

In [ ]:
metrics_all["no_chattn"].keys()

In [ ]:
metrics_all.keys()

In [ ]:
unrolled_metrics_all_groups = defaultdict(lambda: defaultdict(dict))
for run_group, all_metrics_of_run_group in metrics_all.items():
    print(run_group)
    for run_abbrv, all_metrics_of_run_abbrv in all_metrics_of_run_group.items():
        print(run_abbrv)
        for run_name, metrics in all_metrics_of_run_abbrv.items():
            print(run_name)
            systems = metrics.pop("system")
            metrics_unrolled = {k: list(v.values()) for k, v in metrics.items()}
            print(metrics_unrolled.keys())
            unrolled_metrics_all_groups[run_group][run_abbrv][run_name] = (
                metrics_unrolled
            )

In [ ]:
unrolled_metrics_all_combined = {
    **unrolled_metrics_all_groups["chattn"],
    **unrolled_metrics_all_groups["no_chattn"],
}

In [ ]:
unrolled_metrics_all_groups["no_chattn"].keys()

In [ ]:
def get_summary_metrics_dict(unrolled_metrics, metric_name):
    summary_metrics_dict = defaultdict(dict)
    for model_name, metrics_dict in unrolled_metrics.items():
        print(model_name)
        prediction_lengths = list(metrics_dict.keys())
        summary_metrics_dict[model_name]["prediction_lengths"] = prediction_lengths
        means = []
        medians = []
        stds = []
        for prediction_length in prediction_lengths:
            metric_val = metrics_dict[prediction_length][metric_name]
            means.append(np.nanmean(metric_val))
            medians.append(np.nanmedian(metric_val))
            stds.append(np.nanstd(metric_val))
        summary_metrics_dict[model_name]["means"] = means
        summary_metrics_dict[model_name]["medians"] = medians
        summary_metrics_dict[model_name]["stds"] = stds
    return summary_metrics_dict

In [ ]:
def plot_metrics_by_prediction_length(
    metrics_dict, metric_name, show_std_envelope=False
):
    plt.figure(figsize=(5, 4))
    for model_name, metrics in metrics_dict.items():
        plt.plot(
            metrics["prediction_lengths"],
            metrics["medians"],
            marker="o",
            label=model_name,
        )
        std_envelope = np.array(metrics["stds"])
        if show_std_envelope:
            plt.fill_between(
                metrics["prediction_lengths"],
                metrics["means"] - std_envelope,
                metrics["means"] + std_envelope,
                alpha=0.2,
            )
    plt.legend(loc="lower right")
    plt.xlabel("Prediction Length")
    plt.title(metric_name, fontweight="bold")

In [ ]:
run_metrics_dirs_all_groups.keys()

In [ ]:
metric_names_chosen = [
    "mse",
    "mae",
    "smape",
    "r2_score",
    "spearman",
]

In [ ]:
all_metrics_dict = defaultdict(dict)

for run_group in run_metrics_dirs_all_groups.keys():
    all_metrics_dict[run_group] = {
        metrics_name: get_summary_metrics_dict(
            unrolled_metrics_all_groups[run_group], metrics_name
        )
        for metrics_name in metric_names_chosen
    }

In [ ]:
default_colors = plt.cm.tab10.colors

In [ ]:
all_metrics_dict["chattn"]["mse"].keys()

In [ ]:
def plot_all_metrics_by_prediction_length(
    all_metrics_dict: dict[str, dict[str, dict[str, list[float]]]],
    metric_names: list[str],
    runs_to_exclude: list[str],
    metrics_to_show_ste_envelope: list[str] | None = None,
    n_rows: int = 1,
    n_cols: int = 3,
    limit_num_prediction_lengths: int | None = None,
    title: str | None = None,
    custom_colors_dict: dict[str, str] | None = None,
    show_legend: bool = True,
    legend_kwargs: dict[str, Any] = {},
    alpha_val: float = 0.8,
    use_inv_spearman: bool = False,
    legend_height_ratio: float = 0.2,  # New parameter to control legend height
    subplot_size: tuple[float, float] = (4, 4),
):
    num_metrics = len(metric_names)

    if custom_colors_dict is not None:
        run_names_order = list(custom_colors_dict.keys())
        # reorder all_metrics_dict keys according to run_names_order
        for metric_name in all_metrics_dict.keys():
            all_metrics_dict[metric_name] = {
                k: all_metrics_dict[metric_name][k] for k in run_names_order
            }

    # Create figure with gridspec for better control over subplot heights
    if show_legend:
        fig = plt.figure(
            figsize=(
                subplot_size[0] * n_cols,
                subplot_size[1] * n_rows * (1 + legend_height_ratio),
            )
        )
        gs = plt.GridSpec(
            n_rows + 1, n_cols, height_ratios=[1] * n_rows + [legend_height_ratio]
        )
        axes = [
            fig.add_subplot(gs[i // n_cols, i % n_cols]) for i in range(num_metrics)
        ]
    else:
        fig, axes = plt.subplots(
            nrows=n_rows,
            ncols=n_cols,
            figsize=(subplot_size[0] * n_cols, subplot_size[1] * n_rows),
        )
        axes = axes.flatten()

    for i, (ax, metric_name) in enumerate(zip(axes, metric_names)):
        metrics_dict = all_metrics_dict[metric_name]
        for j, (model_name, metrics) in enumerate(metrics_dict.items()):
            if model_name in runs_to_exclude:
                continue
            prediction_lengths = metrics["prediction_lengths"][
                :limit_num_prediction_lengths
            ]
            medians = metrics["medians"][:limit_num_prediction_lengths]
            means = metrics["means"][:limit_num_prediction_lengths]
            stds = metrics["stds"][:limit_num_prediction_lengths]

            if metric_name == "spearman" and use_inv_spearman:
                medians = [1 - x for x in medians]
                means = [1 - x for x in means]

            chosen_color = (
                custom_colors_dict[model_name]
                if custom_colors_dict
                else default_colors[j]
            )
            ax.plot(
                prediction_lengths,
                medians,
                marker="o",
                label=model_name,
                color=chosen_color,
                alpha=alpha_val,
            )
            if (
                metrics_to_show_ste_envelope is not None
                and metric_name in metrics_to_show_ste_envelope
            ):
                # make ste (standard error) envelope
                ste_envelope = np.array(stds) / np.sqrt(len(stds))
                ax.fill_between(
                    prediction_lengths,
                    means - ste_envelope,
                    means + ste_envelope,
                    alpha=0.1,
                    color=chosen_color,
                )
        ax.set_xlabel("Prediction Length")
        ax.set_xticks(prediction_lengths)
        name = metric_name.replace("_", " ")
        if name in ["mse", "mae", "rmse", "mape", "smape"]:
            name = name.upper()
        elif name == "spearman":
            name = "1 - Spearman"
        else:
            name = name.capitalize()
        ax.set_title(name, fontweight="bold")

    if show_legend:
        # Create legend subplot using gridspec
        legend_ax = fig.add_subplot(gs[n_rows, :])
        handles, labels = axes[0].get_legend_handles_labels()
        legend_ax.legend(handles, labels, **legend_kwargs)
        legend_ax.axis("off")

    # Hide any unused subplots
    for ax in axes[num_metrics:]:
        ax.set_visible(False)

    if title is not None:
        plt.suptitle(title, fontweight="bold", fontsize=16, y=1.05)
    plt.tight_layout()
    plt.show()

In [ ]:
# plot_all_metrics_by_prediction_length(
#     all_metrics_dict["chattn"],
#     ["mse", "mae", "smape", "spearman"],
#     # metrics_to_show_ste_envelope=["smape", "pearson", "spearman"],
#     runs_to_exclude=[],
#     metrics_to_show_ste_envelope=[],
#     limit_num_prediction_lengths=None,
#     title="Channel Attention",
#     n_cols=4,
#     show_legend=True,
#     legend_kwargs={
#         "loc": "center",
#         "frameon": True,
#         "ncol": 4,
#         "framealpha": 1.0,
#         "fontsize": 16,
#     },
#     legend_height_ratio=0.4,
# )

In [ ]:
all_metrics_dict_all = {
    metrics_name: {
        **all_metrics_dict["chattn"][metrics_name],
        **all_metrics_dict["no_chattn"][metrics_name],
    }
    for metrics_name in metric_names_chosen
}
plot_all_metrics_by_prediction_length(
    all_metrics_dict_all,
    ["mse", "mae", "smape", "spearman"],
    # metrics_to_show_ste_envelope=["smape", "pearson", "spearman"],
    runs_to_exclude=[],
    metrics_to_show_ste_envelope=[],
    limit_num_prediction_lengths=None,
    title="All Models",
    n_cols=4,
    show_legend=True,
    legend_kwargs={
        "loc": "center",
        "frameon": True,
        "ncol": 3,
        "framealpha": 1.0,
        "fontsize": 16,
    },
    legend_height_ratio=0.4,
)

In [ ]:
from typing import Any

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


def make_aggregate_plot(
    unrolled_metrics: dict[str, dict[int, dict[str, list[float]]]],
    prediction_length: int,
    selected_run_names: list[str] | None = None,
    ylim: tuple[float, float] = (1e-5, 1e5),
    verbose: bool = False,
    metrics_to_include: list[str] = [],
    run_names_to_exclude: list[str] = [],
    use_rescaled_smape: bool = False,
    use_inv_spearman: bool = False,
    title: str = "Metrics",
    fig_kwargs: dict[str, Any] = {},
    legend_kwargs: dict[str, Any] = {},
    title_kwargs: dict[str, Any] = {},
    bar_kwargs: dict[str, Any] = {},
    plot_type: str = "box",  # New parameter: 'box' or 'bar'
    colors: list[str] | None = None,
    sort_by_metric: str | None = None,
    save_path: str | None = None,
):
    if colors is None:
        colors = plt.cm.tab10.colors
    # Extract metrics data for the given prediction_length and run_names
    if selected_run_names is None:
        selected_run_names = list(unrolled_metrics.keys())

    metrics_by_run_name = {
        run_name: unrolled_metrics[run_name][prediction_length]
        for run_name in selected_run_names
    }

    run_names = list(metrics_by_run_name.keys())
    metric_names = list(metrics_by_run_name[run_names[0]].keys())
    metric_names = [name for name in metric_names if name in metrics_to_include]
    run_names = [name for name in run_names if name not in run_names_to_exclude]

    print(f"run_names: {run_names}")
    print(f"metric_names: {metric_names}")

    # Create pretty titles for x-axis tick labels
    metric_names_title = []
    for name in metric_names:
        # Create pretty titles for x-axis tick labels
        if name in ["mse", "mae", "rmse", "mape"]:
            name = name.upper()
        elif name == "smape":
            name = "sMAPE"
        elif name == "spearman":
            name = "1 - Spearman"
        else:
            name = name.capitalize()
        metric_names_title.append(name)

    if fig_kwargs == {}:
        fig_kwargs = {"figsize": (6, 4)}
    plt.figure(**fig_kwargs)

    plot_data = []
    median_data = []
    for i, metric_name in enumerate(metric_names):
        metric_title = metric_names_title[i]
        for run_name in run_names:
            values = metrics_by_run_name[run_name][metric_name]
            if metric_name == "smape" and use_rescaled_smape:
                values = [x / 200 for x in values]
            if metric_name == "spearman" and use_inv_spearman:
                values = [1 - x for x in values]
            median_value = np.nanmedian(values)
            plot_data.extend([(metric_title, v, run_name) for v in values])
            median_data.append((metric_title, median_value, run_name))
            if verbose:
                print(f"{metric_title} - {run_name} median: {median_value}")

    # Create DataFrame for use with seaborn
    df = pd.DataFrame(plot_data, columns=["Metric", "Value", "Run"])

    # Set the order of metrics to match the original order in metric_names_title
    df["Metric"] = pd.Categorical(
        df["Metric"], categories=metric_names_title, ordered=True
    )

    # Sort runs based on the specified metric or overall median
    if sort_by_metric is not None:
        # Find the pretty title for the specified metric
        sort_metric_title = None
        for i, metric_name in enumerate(metric_names):
            if metric_name.lower() == sort_by_metric.lower():
                sort_metric_title = metric_names_title[i]
                break

        if sort_metric_title is None:
            print(
                f"Warning: Metric '{sort_by_metric}' not found. Using overall median for sorting."
            )
            # Calculate the median for each run (across all metrics)
            median_values = df.groupby("Run")["Value"].median().reset_index()
        else:
            # Filter data for the specific metric and calculate median
            metric_df = df[df["Metric"] == sort_metric_title]
            median_values = metric_df.groupby("Run")["Value"].median().reset_index()
    else:
        # Calculate the median for each run (across all metrics)
        median_values = df.groupby("Run")["Value"].median().reset_index()

    # Sort the medians
    sorted_medians = median_values.sort_values("Value")

    # Create a categorical type with the sorted run names to preserve the order
    df["Run"] = pd.Categorical(
        df["Run"], categories=sorted_medians["Run"].tolist(), ordered=True
    )

    # Sort the original dataframe
    df = df.sort_values("Run")

    # Choose plot type based on parameter
    if plot_type == "box":
        # Plot box plot
        sns.boxplot(
            data=df,
            x="Metric",
            y="Value",
            hue="Run",
            dodge=True,
            width=0.8,
            fliersize=0,  # Don't show outlier points
            palette=colors[: len(run_names)],
            medianprops={"linewidth": 2.5, "solid_capstyle": "butt"},
            saturation=0.6,
        )
    elif plot_type == "bar":
        if bar_kwargs == {}:
            bar_kwargs = {"estimator": "median"}
        # Plot bar plot
        sns.barplot(
            data=df,
            x="Metric",
            y="Value",
            hue="Run",
            palette=colors[: len(run_names)],
            saturation=0.6,
            **bar_kwargs,
        )
    else:
        raise ValueError(f"Unsupported plot_type: {plot_type}")

    plt.ylim(ylim)
    plt.ylabel("")
    plt.xlabel("")
    plt.xticks(fontweight="bold")
    plt.legend(**legend_kwargs)
    plt.title(title, fontweight="bold", **title_kwargs)
    plt.xticks(rotation=15)  # Optional: rotates x-tick labels for readability
    plt.tight_layout()
    if save_path is not None:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        plt.savefig(save_path, bbox_inches="tight")
    plt.show()

In [ ]:
unrolled_metrics_all_groups["chattn"].keys()

In [ ]:
unrolled_metrics_all_combined.keys()

In [ ]:
light_gray = (0.7, 0.7, 0.7)
bar_colors = list(plt.cm.tab20c.colors[:8]) + [light_gray]
print(len(bar_colors))

In [ ]:
make_aggregate_plot(
    unrolled_metrics_all_combined,
    128,
    selected_run_names=None,
    ylim=(0, 1),
    metrics_to_include=["mse", "mae", "smape", "spearman"],
    use_rescaled_smape=True,
    use_inv_spearman=True,
    title="Zeroshot Metrics for Ablations",
    legend_kwargs={
        "loc": "upper right",
        "fontsize": 6,
        "frameon": True,
        "framealpha": 1.0,
    },
    plot_type="box",
    colors=bar_colors,
    sort_by_metric="smape",
    save_path=None,
)

In [ ]:
def make_aggregate_plot_v2(
    unrolled_metrics: dict[str, dict[int, dict[str, list[float]]]],
    prediction_length: int,
    metric_to_plot: str = "smape",  # Default to smape
    selected_run_names: list[str] | None = None,
    ylim: tuple[float, float] | None = None,  # Changed to None as default
    verbose: bool = False,
    run_names_to_exclude: list[str] = [],
    use_rescaled_smape: bool = False,
    use_inv_spearman: bool = False,
    title: str | None = None,
    fig_kwargs: dict[str, Any] = {},
    title_kwargs: dict[str, Any] = {},
    colors: list[str] | None = None,
    sort_runs: bool = False,
    save_path: str | None = None,
    order_by_metric: str | None = None,
    ylabel_fontsize: int = 8,
    show_xlabel: bool = True,
    show_legend: bool = False,
    legend_kwargs: dict[str, Any] = {},
    alpha_val: float = 0.8,
    box_percentile_range: tuple[int, int] = (25, 75),
    whisker_percentile_range: tuple[float, float] = (0, 90),
) -> list[mpatches.Patch] | None:
    # Set default figure size if not provided
    if fig_kwargs == {}:
        fig_kwargs = {"figsize": (3, 5)}  # Wider figure to accommodate run names

    if colors is None:
        colors = plt.cm.tab10.colors  # type: ignore

    # Extract metrics data for the given prediction_length and run_names
    if selected_run_names is None:
        selected_run_names = list(unrolled_metrics.keys())

    # Filter out excluded run names
    run_names = [
        name for name in selected_run_names if name not in run_names_to_exclude
    ]

    if len(run_names) == 0:
        print("No run names to plot after exclusions!")
        return

    plt.figure(**fig_kwargs)
    # Collect data for plotting
    plot_data = []

    # Add a dictionary to store ordering metric values if needed
    ordering_metric_data = {}

    for run_name in run_names:
        try:
            # Check if this run has data for the specified prediction length
            if prediction_length not in unrolled_metrics[run_name]:
                print(
                    f"Warning: prediction_length {prediction_length} not found for {run_name}"
                )
                continue

            # Check if this run has the specified metric
            if metric_to_plot not in unrolled_metrics[run_name][prediction_length]:
                print(f"Warning: metric '{metric_to_plot}' not found for {run_name}")
                continue

            values = unrolled_metrics[run_name][prediction_length][metric_to_plot]

            # Process values based on metric type
            if metric_to_plot == "smape" and use_rescaled_smape:
                values = [x / 200 for x in values]
            if metric_to_plot == "spearman" and use_inv_spearman:
                values = [1 - x for x in values]

            # Filter out NaN values
            values = [v for v in values if not np.isnan(v)]

            if len(values) == 0:
                print(f"Warning: All values for {run_name} are NaN")
                continue

            median_value = np.median(values)
            plot_data.extend([(run_name, v) for v in values])

            # If we need to order by a different metric, collect that data too
            if order_by_metric is not None and order_by_metric != metric_to_plot:
                if order_by_metric in unrolled_metrics[run_name][prediction_length]:
                    order_values = unrolled_metrics[run_name][prediction_length][
                        order_by_metric
                    ]

                    # Apply same processing as we would for the plotting metric
                    if order_by_metric == "smape" and use_rescaled_smape:
                        order_values = [x / 200 for x in order_values]
                    if order_by_metric == "spearman" and use_inv_spearman:
                        order_values = [1 - x for x in order_values]

                    # Filter out NaN values
                    order_values = [v for v in order_values if not np.isnan(v)]

                    if order_values:
                        ordering_metric_data[run_name] = np.median(order_values)

            if verbose:
                print(f"{run_name} median {metric_to_plot}: {median_value}")

        except Exception as e:
            print(f"Error processing {run_name}: {e}")

    df = pd.DataFrame(plot_data, columns=["Run", "Value"])

    # Determine run order based on specified criteria
    if order_by_metric is not None and ordering_metric_data:
        # Sort runs based on the ordering metric's median values
        run_order = [
            run for run, _ in sorted(ordering_metric_data.items(), key=lambda x: x[1])
        ]
        # Only include runs that are in our dataframe
        run_order = [run for run in run_order if run in df["Run"].unique()]
        # Set categorical order for Run column
        df["Run"] = pd.Categorical(df["Run"], categories=run_order, ordered=True)
    elif sort_runs:
        # Use the existing sort_runs logic if order_by_metric isn't specified
        median_by_run = df.groupby("Run")["Value"].median().sort_values()
        run_order = median_by_run.index.tolist()
        df["Run"] = pd.Categorical(df["Run"], categories=run_order, ordered=True)

    metric_title = metric_to_plot
    if metric_to_plot in ["mse", "mae", "rmse", "mape"]:
        metric_title = metric_to_plot.upper()
    elif metric_to_plot == "smape":
        metric_title = "sMAPE"
    elif metric_to_plot == "spearman":
        metric_title = "1 - Spearman" if use_inv_spearman else "Spearman"
    else:
        metric_title = metric_to_plot.capitalize()

    # Create a custom boxplot with the specified percentile ranges
    ax = plt.gca()

    unique_runs = (
        df["Run"].unique()
        if not isinstance(df["Run"].dtype, pd.CategoricalDtype)
        else df["Run"].cat.categories
    )

    for i, run in enumerate(unique_runs):
        run_data = df[df["Run"] == run]["Value"].to_numpy()
        if len(run_data) == 0:
            continue

        # Calculate the percentiles
        lower_box, upper_box = np.percentile(run_data, box_percentile_range)
        lower_whisker, upper_whisker = np.percentile(run_data, whisker_percentile_range)
        median_val = np.median(run_data)

        # Plot the custom boxplot
        color = colors[i % len(colors)]  # type: ignore

        # Box
        box = plt.Rectangle(
            (i - 0.3, lower_box),
            0.6,
            upper_box - lower_box,
            fill=True,
            facecolor=color,
            alpha=alpha_val,
            linewidth=1,
            edgecolor="black",
            zorder=5,
        )
        ax.add_patch(box)

        # Median line
        ax.hlines(
            median_val, i - 0.3, i + 0.3, colors="black", linewidth=2.5, zorder=10
        )

        # Whiskers
        ax.vlines(
            i,
            lower_box,
            lower_whisker,
            colors="black",
            linestyle="-",
            linewidth=1,
            zorder=5,
        )
        ax.vlines(
            i,
            upper_box,
            upper_whisker,
            colors="black",
            linestyle="-",
            linewidth=1,
            zorder=5,
        )

        # Caps on whiskers
        ax.hlines(
            lower_whisker, i - 0.15, i + 0.15, colors="black", linewidth=1, zorder=5
        )
        ax.hlines(
            upper_whisker, i - 0.15, i + 0.15, colors="black", linewidth=1, zorder=5
        )

    # Set y-limits if provided
    if ylim:
        plt.ylim(ylim)

    # Format the plot
    plt.ylabel(metric_title, fontweight="bold", fontsize=ylabel_fontsize)
    plt.xlabel("")
    if show_xlabel:
        # Format x-axis labels
        plt.xticks(
            range(len(unique_runs)),
            unique_runs,
            rotation=45,
            ha="right",
            fontsize=5,
            fontweight="bold",
        )
    else:
        plt.xticks([])

    # Set the title
    if title is not None:
        title_with_metric = f"{title}: {metric_title}" if title == "Metrics" else title
        plt.title(title_with_metric, fontweight="bold", **title_kwargs)

    # Ensure plot is properly displayed
    plt.tight_layout()

    # Get the unique run names in the correct order
    if isinstance(df["Run"].dtype, pd.CategoricalDtype):
        runs = df["Run"].cat.categories.tolist()
    else:
        runs = df["Run"].unique().tolist()

    # Create custom legend handles
    legend_handles = [
        mpatches.Patch(color=colors[i % len(colors)], label=run, alpha=alpha_val)  # type: ignore
        for i, run in enumerate(runs)
    ]

    if show_legend:
        plt.legend(handles=legend_handles, **legend_kwargs)

    # Save if path provided
    if save_path is not None:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        plt.savefig(save_path, bbox_inches="tight")

    plt.show()
    return legend_handles

In [ ]:
selected_pred_length = 128

In [ ]:
df = make_aggregate_plot_v2(
    unrolled_metrics=unrolled_metrics_all_combined,
    prediction_length=selected_pred_length,
    metric_to_plot="smape",  # Specify which metric to plot
    sort_runs=True,  # Optionally sort runs by their metric values
    colors=bar_colors,
    title=None,
    title_kwargs={"fontsize": 10},
    save_path="ablations_figs/smape_128.pdf",
    ylabel_fontsize=12,
    show_xlabel=False,
    box_percentile_range=(40, 60),
    whisker_percentile_range=(25, 75),
    alpha_val=0.8,
)

In [ ]:
make_aggregate_plot_v2(
    unrolled_metrics=unrolled_metrics_all_combined,
    prediction_length=selected_pred_length,
    metric_to_plot="spearman",  # Specify which metric to plot
    sort_runs=True,  # Optionally sort runs by their metric values
    colors=bar_colors,
    title=None,
    title_kwargs={"fontsize": 10},
    use_inv_spearman=True,
    order_by_metric="smape",
    save_path="ablations_figs/spearman_128.pdf",
    ylabel_fontsize=12,
    show_xlabel=False,
    box_percentile_range=(40, 60),
    whisker_percentile_range=(25, 75),
    alpha_val=0.8,
)

In [ ]:
make_aggregate_plot_v2(
    unrolled_metrics=unrolled_metrics_all_combined,
    prediction_length=selected_pred_length,
    metric_to_plot="mae",  # Specify which metric to plot
    sort_runs=True,  # Optionally sort runs by their metric values
    colors=bar_colors,
    title=None,
    title_kwargs={"fontsize": 10},
    use_inv_spearman=True,
    order_by_metric="smape",
    save_path="ablations_figs/mae_128.pdf",
    ylabel_fontsize=12,
    show_xlabel=False,
    box_percentile_range=(40, 60),
    whisker_percentile_range=(25, 75),
    alpha_val=0.8,
)

In [ ]:
legend_handles = make_aggregate_plot_v2(
    unrolled_metrics=unrolled_metrics_all_combined,
    prediction_length=selected_pred_length,
    metric_to_plot="mse",  # Specify which metric to plot
    sort_runs=True,  # Optionally sort runs by their metric values
    colors=bar_colors,
    title=None,
    title_kwargs={"fontsize": 10},
    use_inv_spearman=True,
    order_by_metric="smape",
    save_path="ablations_figs/mse_128.pdf",
    ylabel_fontsize=12,
    show_xlabel=False,
    show_legend=False,
    legend_kwargs={
        "loc": "upper left",
        "frameon": True,
        "ncol": 1,
        "framealpha": 0.8,
        # "prop": {"weight": "bold", "size": 5},
        "prop": {"size": 6.8},
    },
    box_percentile_range=(40, 60),
    whisker_percentile_range=(25, 75),
    alpha_val=0.8,
)

In [ ]:
plt.figure(figsize=(4, 0.6))
# Add the legend
plt.legend(
    handles=legend_handles,
    loc="center",
    frameon=True,
    ncol=3,
    framealpha=1.0,
)
plt.xticks([])
plt.yticks([])
plt.tight_layout(pad=0)
plt.savefig("ablations_figs/ablations_legend.pdf", bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
legend_handles[0].get_label(), legend_handles[0].get_facecolor()

In [ ]:
custom_colors_dict = {}
for i, patch in enumerate(legend_handles):
    color = patch.get_facecolor()
    hex_color = mcolors.rgb2hex(color)
    run_name = patch.get_label()
    print(run_name, hex_color)
    custom_colors_dict[run_name] = hex_color

In [ ]:
all_metrics_dict_all["smape"].keys()

In [ ]:
plot_all_metrics_by_prediction_length(
    all_metrics_dict_all,
    ["mse", "mae", "smape", "spearman"],
    # metrics_to_show_ste_envelope=["smape", "spearman"],
    runs_to_exclude=[],
    limit_num_prediction_lengths=None,
    title=None,
    n_rows=1,
    n_cols=4,
    custom_colors_dict=custom_colors_dict,
    show_legend=True,
    legend_kwargs={
        "loc": "center",
        "frameon": True,
        "ncol": 3,
        "framealpha": 1.0,
        "fontsize": 16,
    },
    legend_height_ratio=0.4,
    alpha_val=0.8,
    use_inv_spearman=True,
)

In [ ]:
# all_run_names = [
#     "chattn + mlm + embedding",
#     "chattn + mlm",
#     "chattn + embedding",
#     "chattn",
#     "univariate (wider) + embedding",
#     "univariate (wider)",
#     "univariate (deeper)",
#     "univariate + mlm + embedding",
#     "univariate + mlm",
# ]

run_names_to_exclude = [
    # "chattn + mlm + embedding",
    # "chattn + mlm",
    # "chattn + embedding",
    # "chattn",
    "univariate (wider) + embedding",
    "univariate (wider)",
    "univariate (deeper)",
    "univariate + mlm + embedding",
    "univariate + mlm",
]

In [ ]:
plot_all_metrics_by_prediction_length(
    all_metrics_dict_all,
    ["mse", "mae", "smape", "spearman"],
    # metrics_to_show_ste_envelope=["smape", "spearman"],
    runs_to_exclude=run_names_to_exclude,
    limit_num_prediction_lengths=None,
    title=None,
    n_rows=1,
    n_cols=4,
    # custom_colors_dict=custom_colors_dict,
    show_legend=True,
    legend_kwargs={
        "loc": "center",
        "frameon": True,
        "ncol": 4,
        "framealpha": 1.0,
        "fontsize": 16,
    },
    legend_height_ratio=0.4,
    alpha_val=0.8,
    use_inv_spearman=True,
)

In [ ]:
run_names_to_exclude = [
    # "chattn + mlm + embedding",
    # "chattn + mlm",
    # "chattn + embedding",
    # "chattn",
    "univariate (wider) + embedding",
    "univariate (wider)",
    "univariate (deeper)",
    "univariate + mlm + embedding",
    "univariate + mlm",
]

In [ ]:
plot_all_metrics_by_prediction_length(
    all_metrics_dict_all,
    ["smape", "spearman"],
    metrics_to_show_ste_envelope=["smape", "spearman"],
    runs_to_exclude=run_names_to_exclude,
    limit_num_prediction_lengths=None,
    title=None,
    n_rows=1,
    n_cols=2,
    custom_colors_dict=custom_colors_dict,
    show_legend=True,
    legend_kwargs={
        "loc": "center",
        "frameon": True,
        "ncol": 2,
        "framealpha": 1.0,
        "fontsize": 16,
    },
    legend_height_ratio=0.4,
    alpha_val=0.8,
    use_inv_spearman=True,
    subplot_size=(4, 4),
)

In [ ]:
run_names_to_exclude = [
    # "chattn + mlm + embedding",
    # "chattn + mlm",
    "chattn + embedding",
    "chattn",
    "univariate (wider) + embedding",
    "univariate (wider)",
    "univariate (deeper)",
    "univariate + mlm + embedding",
    "univariate + mlm",
]

In [ ]:
smape_metrics_dict = all_metrics_dict_all["smape"]
fig = plt.figure(figsize=(4, 4))

colors = plt.cm.tab20c.colors
markers = ["o", "s", "D", "P"]
linestyles = ["-", "--", "-.", ":"]
for j, (model_name, metrics) in enumerate(smape_metrics_dict.items()):
    if model_name in run_names_to_exclude:
        continue
    prediction_lengths = metrics["prediction_lengths"]
    medians = metrics["medians"]
    means = metrics["means"]
    stds = metrics["stds"]

    plt.plot(
        prediction_lengths,
        medians,
        marker=markers[j],
        label=model_name,
        color=colors[j],
        alpha=1,
        linestyle=linestyles[j],
    )
    # make ste (standard error) envelope
    ste_envelope = np.array(stds) / np.sqrt(len(stds))
    plt.fill_between(
        prediction_lengths,
        means - ste_envelope,
        means + ste_envelope,
        alpha=0.1,
        color=colors[j],
    )
plt.xlabel("Prediction Length", fontweight="bold")
plt.xticks(prediction_lengths)
plt.title("sMAPE", fontweight="bold")
plt.legend(loc="lower right", frameon=True, framealpha=1.0, fontsize=10)
plt.tight_layout()
plt.savefig("ablations_figs/smape_embedding_ablation.pdf", bbox_inches="tight")
plt.show()